# Java Keytool

The `keytool` command is bundled with Java and has many options which are not always easy to remember.

Included is a set of common tasks:

- Generate a new Certificate Authority (CA)
- Generate a new Server Certificate from the CA (TODO)
- Generate a New Java Truststore (JKS) with an Existing CA
- Import a .crt into a Custom Truststore (TODO)
- Import `cacerts` into a Custom Truststore
- 
  

## Initialize Secrets

Initialize secrets from the local `.env` file:

In [ ]:
source .env
echo $MY_SECRET

## Confirm the Java Installation

Verify that a Java is present. If not, consider using [SDKMAN](https://sdkman.io/) as it will greatly simplify managing and updating Java. Java is currently on version 21 (if you are still on Java 8, please upgrade!).

In [ ]:
echo $JAVA_HOME

In [ ]:
java -version

Every Java installation comes with a CLI tool called `keytool`. This command is used to help generate and manage certificates, certificate authorities and passwords.

In [ ]:
keytool --help

## The Java `cacerts` Keystore

Java comes bundled with a cacert file already populated with trusted certificate authorities (CA). This file can be inspected and have its contents listed to verify existing trusted CAs (trustedCertEntry). It is not usually recommended to modify this file as it could introduce a security vulnerability to Java - although in a corporate environment, it is possible that it has already been modified to include internal CAs used for intranet access.

Note the default Java password is `changeit` as that may also have been changed by corporate IT.

In [ ]:
keytool -cacerts -storepass changeit -list

## Tasks

### >> Generate a new Certificate Authority (CA)

A certificate authority is what is used to sign certificates for server hosts.

When configuring hosts within a private network, it is very common to use a self-signed certificate authority (or custom CA) to provision new host certificates. The `keytool` command can be used to create a new CA with the `genkeypair` option and automatically add it to a new Java key store:

In [ ]:
keytool -keystore ca-keystore.jks -storepass change1t -genkeypair -alias cn_undertree_private_key -keyalg RSA -keysize 2048 -validity 3650 -dname "CN=UnderTree.io, OU=Home, O=Office, L=Cincinnati, S=OH, C=US"

Confirm that the private key and certificate are contained in the new keystore using `list`:

In [ ]:
keytool -keystore ca-keystore.jks -storepass change1t -list

Export the public certificate of the CA (this is the file that can be imported into other truststores to accept server certificates later generated by this CA):

In [ ]:
keytool -keystore ca-keystore.jks -storepass change1t -exportcert -alias undertree-ca-key -file undertree-ca.crt

In [ ]:
keytool -printcert -file undertree-ca.crt

### >> Generate a New Java Truststore (JKS) with an Existing CA

The following command will create a Java Key Store (JKS) file and import a certificate authority (CA) from the filesystem. This command uses `-noprompt` to avoid the console prompt to accept:

In [ ]:
keytool -keystore local-truststore.jks -storepass change1t -importcert -trustcacerts -file undertree-ca.crt -alias cn_undertree_selfsigned_ca -noprompt

A new JKS file should have been created called `local-truststore.jks` with the `undertree-ca.crt` added as trustedCertEntity. This allows it to be used to validate any server presenting a certificate generated from the undertree-ca. Use the `-list` option to verify:

In [ ]:
keytool -keystore local-truststore.jks -storepass change1t -list 

Any Java application configured to use this truststore will allow connections to ONLY hosts that present certificates generated from the `cn_undertree_selfsigned_ca` (the `.crt` that was just imported).

## >> Import `cacerts` into the Local Truststore

If desired, the default `cacerts` trusted cert entries can also be added to this local truststore. This would allow the application to also be able to connect hosts signed by any of the external trusted authorities:

In [ ]:
keytool -importkeystore -srckeystore $JAVA_HOME/lib/security/cacerts -destkeystore local-truststore.jks -srcstorepass change1t -deststorepass change1t

In [ ]:
# Example way to override any Java application
java -Djavax.net.ssl.trustStore=/path/to/local-truststore.jks -Djavax.net.ssl.trustStorePassword=change1t -jar yourapp.jar